In [6]:
using Plots
using Statistics
using LinearAlgebra
using JSON

include("readclassjson.jl");

In [3]:
# ; wget http://ee263.stanford.edu/yayfor21/neural_network.json -P data

## Problem 1, g)

In [217]:
r = 10
m = 15e3
k = 15e4

150000.0

## Problem 2 a)

In [14]:
A = [1 2 ; 3 4 ; 5 6]
B = [ 7 8 ; 1 1 ; 4 2 ; 9 1];

In [22]:
ma = size(A)[1]
mb = size(B)[1]
n = size(A)[2]

2

In [15]:
block = [A ; B];

In [101]:
svd1 = svd(block, full=true)
U11 = svd1.U[1:ma, 1:n]
U21 = svd1.U[(ma+1):end, 1:n]
V = svd1.V
sigma = diagm(svd1.S)

2×2 Array{Float64,2}:
 16.5915  0.0
  0.0     5.72044

In [28]:
svd2 = svd(U21, full=true)
Ub = svd2.U
V3t = svd2.Vt
V3 = svd2.V
sigmab = diagm(svd2.S)

2×2 Array{Float64,2}:
 0.997115  0.0
 0.0       0.733151

In [37]:
Ua = svd1.U[1:ma, (n+1):(ma+n)]

3×3 Array{Float64,2}:
 -0.362756  -0.509232  -0.0732378
 -0.40087   -0.504645  -0.0518877
  0.790089  -0.279559  -0.034824

In [44]:
U11 * V3 * Ua[:,1:2]'

3×3 Array{Float64,2}:
 -0.0755123  -0.0721104  -0.106522
 -0.1811     -0.178822   -0.114893
 -0.286688   -0.285533   -0.123264

In [90]:
qr1 = qr(U11 * V3)
qrQ = qr1.Q
Ua = reduce(hcat, Ua[:,i] for i in [1,2,3])
sigmaa = qr1.R

2×2 Array{Float64,2}:
 0.0758998   1.13798e-15
 0.0        -0.680066

In [91]:
sigmaa[:,2] = sigmaa[:,2] .* -1
Ua[:,2] = Ua[:,2] .* -1

3-element Array{Float64,1}:
 0.286945838426678
 0.5393238577758741
 0.79170187712507

In [106]:
isapprox(U11 * V3, Ua * [sigmaa ; zeros(1,2)])

true

In [96]:
sigmaa

2×2 Array{Float64,2}:
 0.0758998  -1.13798e-15
 0.0         0.680066

In [103]:
sigmab

2×2 Array{Float64,2}:
 0.997115  0.0
 0.0       0.733151

In [104]:
Ua

3×3 Array{Float64,2}:
 -0.8666    0.286946   0.408248
 -0.206066  0.539324  -0.816497
  0.454468  0.791702   0.408248

In [105]:
Ub

4×4 Array{Float64,2}:
 0.0848732   0.968482  -0.2213    -0.0765912
 0.0251397   0.117052   0.184262   0.975558
 0.282768    0.169974   0.922173  -0.20186
 0.955095   -0.139467  -0.258206   0.040891

In [102]:
X = V * inv(sigma) * V3

2×2 Array{Float64,2}:
  0.115909   -0.0235084
 -0.0908419   0.109325

## Problem 4 c)

In [7]:
nn = readclassjson("data/neural_network.json")

Dict{Any,Any} with 7 entries:
  "y"  => [1, 1, 1, 0, 1, 0, 1, 0, 0, 1  …  1, 0, 1, 1, 1, 0, 1, 1, 1, 0]
  "X"  => [3.09019 0.0951662; -0.181169 3.54493; … ; 3.19208 1.60698; 2.51807 -…
  "m"  => 100
  "N"  => 10
  "u"  => [-0.927326, -1.94019, 0.205306, -0.702231, 1.37903, -0.287253, 1.4613…
  "mu" => 1.0
  "n"  => 2

In [27]:
X = nn["X"]
y = nn["y"]
u = nn["u"];
n = 2
N = 10
mu = 1
m = 100;

Tried to write this in the alternate form first, solving directly for u. Ran out of time debugging

In [9]:
function f(X, u)
    u_shaped = reshape(u, 2, 10)
    sum(X * u_shaped, dims=2).^2
end

function loss(u)
    norm(f(X, u) .- y)
end

function jacobian(u)
    fit = f(X, u)
    u_shaped = reshape(u, 2, 10)
    du =  2 .* X .* sum(X * u_shaped, dims=2)
    du
end

jacobian (generic function with 1 method)

In [22]:
function fitter(u0, iters=100)
    u = u0
    rmse_vec = zeros(iters)
    for i = 1:iters
        Jk = jacobian(u)
        Ak = Jk
        u_shaped = reshape(u, 2, 10)
        bk = Jk * u_shaped .- Ak'u_shaped
        
        u_k1 = inv(Ak'*Ak + mu .* I) * Ak' * bk

        rmse_vec[i] = loss(u)

        u = u_k1[:]
        
    end
    return(u, rmse_vec)
end

fitter (generic function with 2 methods)

In [23]:
Jk = jacobian(u)
Ak = Jk
u_shaped = reshape(u, 2, 10)
bk = Jk * u_shaped .- loss(u)


100×10 Array{Float64,2}:
 -2170.85  -2238.26  -2306.58  -2311.1   …  -2213.62  -2245.52  -2248.78
 -2055.18  -2162.73  -2195.08  -2183.36     -2223.07  -2275.42  -2260.92
 -1992.46  -2253.14  -2502.87  -2516.77     -2176.66  -2300.28  -2309.18
 -2228.97  -2231.69  -2237.81  -2238.82     -2226.34  -2227.56  -2228.55
 -1823.41  -2137.6   -2325.48  -2314.28     -2192.5   -2343.65  -2325.29
 -2222.57  -2226.25  -2228.08  -2227.86  …  -2227.36  -2229.13  -2228.82
 -2111.45  -2168.95  -2163.75  -2151.93     -2230.45  -2258.87  -2245.33
 -2227.0   -2227.3   -2227.19  -2227.11     -2227.72  -2227.86  -2227.77
 -2179.19  -2223.81  -2258.84  -2259.32     -2220.74  -2242.05  -2241.59
 -2210.06  -2226.88  -2240.71  -2241.04     -2224.92  -2232.95  -2232.93
 -1647.97  -2121.91  -2433.51  -2423.6   …  -2168.05  -2395.52  -2375.07
 -2113.13  -2218.09  -2300.06  -2301.06     -2211.46  -2261.59  -2260.4
 -2221.78  -2226.62  -2229.8   -2229.7      -2227.07  -2229.39  -2229.19
     ⋮                     

In [29]:
output, rmse = fitter(u)

([-2.534460172040803e142, -2.8098488715697473e142, -2.534460172040803e142, -2.8098488715697473e142, -2.534460172040803e142, -2.8098488715697473e142, -2.534460172040803e142, -2.8098488715697473e142, -2.534460172040803e142, -2.8098488715697473e142, -2.534460172040803e142, -2.8098488715697473e142, -2.534460172040803e142, -2.8098488715697473e142, -2.534460172040803e142, -2.8098488715697473e142, -2.534460172040803e142, -2.8098488715697473e142, -2.534460172040803e142, -2.8098488715697473e142], [2227.683873922618, 1.6044499268096676e6, 1.1569608248595848e9, 8.342794963375057e11, 6.015953719039051e14, 4.338078462713747e17, 3.128169801091256e20, 2.25570984678269e23, 1.6265827101512727e26, 1.1729218262431723e29  …  3.687188034479333e260, 2.658815500812296e263, 1.9172604709208955e266, 1.3825283146697423e269, 9.969352468553668e271, 7.188857370056749e274, 5.183854262353792e277, 3.738055108069484e280, 2.6954955297334548e283, 1.943710282689063e286])

In [30]:
rmse

100-element Array{Float64,1}:
 2227.683873922618
    1.6044499268096676e6
    1.1569608248595848e9
    8.342794963375057e11
    6.015953719039051e14
    4.338078462713747e17
    3.128169801091256e20
    2.25570984678269e23
    1.6265827101512727e26
    1.1729218262431723e29
    8.457889057173573e31
    6.098947577144465e34
    4.397924978361769e37
    ⋮
    7.091032130380002e254
    5.113312900971899e257
    3.687188034479333e260
    2.658815500812296e263
    1.9172604709208955e266
    1.3825283146697423e269
    9.969352468553668e271
    7.188857370056749e274
    5.183854262353792e277
    3.738055108069484e280
    2.6954955297334548e283
    1.943710282689063e286

In [31]:
output

20-element Array{Float64,1}:
 -2.534460172040803e142
 -2.8098488715697473e142
 -2.534460172040803e142
 -2.8098488715697473e142
 -2.534460172040803e142
 -2.8098488715697473e142
 -2.534460172040803e142
 -2.8098488715697473e142
 -2.534460172040803e142
 -2.8098488715697473e142
 -2.534460172040803e142
 -2.8098488715697473e142
 -2.534460172040803e142
 -2.8098488715697473e142
 -2.534460172040803e142
 -2.8098488715697473e142
 -2.534460172040803e142
 -2.8098488715697473e142
 -2.534460172040803e142
 -2.8098488715697473e142